# FIT simulations

### Little bit of theory:

**Transfer entropy**: the mutual information between $Y_{pres}$ and $X_{past}$, conditioned on $Y_{past}$

$$ TE(X\rightarrow Y) = I(X_{past}; Y_{pres}|Y_{past}) $$

where

$$ I(S=s,X_i) = \sum_{x_i \in X_i} p(x_i|s)\,\log\frac{p(s|x_i)}{p(s)}$$

is the specific information that the source $X_i$ carries about a specific outcome of the target variable $s\in S$.

**FIT**:

$$ FIT = \text{min}[SUI(S: X_{past}, Y_{pres} \ Y_{past} ),\, SUI(Y_{pres}: X_{past}, S\ Y_{past})] $$

where 
- $SUI(S: X_{past}, Y_{pres} \ Y_{past} )$ is the information about $S$ that $X_{past}$ shares with $Y_{pres}$ and it's unique with respect to $Y_{past}$ and it's defined as the difference between the shared information that $X_{past}, Y_{pres}$ carry about $S$ and the shared information that $X_{past}, Y_{pres}$ and $Y_{past}$ carry about $S$
- $SUI(Y_{pres}: X_{past}, S\ Y_{past})$ is the information about $Y_{pres}$ that $X_{past}$ shares with $S$ and it's unique with respect to $Y_{past}$

Moreover

$$ SUI(S:X_1,X_2) = \sum_{s\in S} p(s) \text{min}_{X_i \in \{ X_1, X_2\} } I(S=s,X_i) $$

is the shared information that $X_1$ and $X_2$ carry about $S$.

**Sender activity**: 2D variable

$$X(t)_{stim} = S(t)(1+N(0, \sigma_{stim}))$$

where $S(t)$ is a step function equal to the value of the stimulus $s \in [1,4]$ during the time window $[200,250]\,ms$.

$$X(t)_{noise} = N(0,\sigma)$$

**Receiver activity**: 1D variable

$$Y(t) = W_{stim}X_{stim}(t-\delta) + W_{noise}X_{noise}(t-\delta) + N(0,\sigma)$$

where the **delay** $\delta$ is chosen randomly from a uniform distribution in $[40,60]\,ms$ in step of $10\,ms$, moreover $\sigma = 2$ and $\sigma_{stim} = \sigma/5 = 0.4$.

**Numerical computation of FIT**: discretization of neural activity into a number R of equipopulated bins and empirical computation of the occurence frequency of each binned response across all available trials.

FIT/TE are computed at the first time instant in which $Y$ received information from $X$.

Total of 50 simulation with 500 trials per stimulus each one.

### WHAT TO DO:

- **Fig. 2A** - evaluation of FIT and TE for each value of $W_{noise}$ and $W_{stim}$
- **Fig. 2B** - FIT and TE as a function of time with $W_{stim}=0.5$ and $W_{noise}=1$ (FIT/TE values computed at all points and averaged over delays to obtain temporal profiles of transmitted information)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from sim2ABroutine import sim2ABroutine

In [2]:
# simulation parameters

nTrials_per_stim = 5      # number of trials for each simulation
simReps = 2                # number of simulations
nShuff = 10                 # number of permutations
w_sig = np.linspace(0, 1, num=11)      # signal weights for Y computation
w_noise = np.linspace(0, 1, num=11)    # noise weights for Y computation
stdX_noise = 2                      # std of gaussian noise in X_noise
stdY = 2                            # std of gaussian noise in Y
ratio = 0.2                         # ratio between stdX_sig and stdX_noise
stdX_sig = ratio * stdX_noise       # std of gaussian noise in X_signal

# global parameters
simLen = 60                             # simulation length in units of 10 ms
stimWin = [30, 35]                      # X stimulus encoding window in units of 10 ms
delays = np.linspace(4, 6, num=3, dtype=int)
delay_max = 10                          # Maximum Computed Delay
n_binsS = 4                             # number of stimulus values
n_binsX = 3
n_binsY = 3
eps = 1e-52                             # infinitesimal value
nTrials = nTrials_per_stim * n_binsS

In [3]:
# files to save the simulation results
file_2AB = 'sim_file_2AB.npz'

# load 2A simulation file if present, otherwise perform simulations
if os.path.exists(file_2AB):

    loadnpz_2AB = np.load(file_2AB)

    te_A = loadnpz_2AB['te_A']
    dfi_A = loadnpz_2AB['dfi_A']
    fit_A = loadnpz_2AB['fit_A']

    # add B code

else:
    fit_A, te_A, dfi_A, fit_B, te_B, dfi_B = sim2ABroutine()
    np.savez(file_2AB, fit_A=fit_A, te_A=te_A, dfi_A=dfi_A, fit_B=fit_B, te_B=te_B, dfi_B=dfi_B)


 Starting part A 

Simulation number:  0 

Simulation number:  1 


 Part A finished 


 Starting part B 

Simulation number:  0 

Simulation number:  1 



KeyboardInterrupt: 

In [ ]:
# heatmaps plot

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,8), squeeze = False)

fitvalues = np.mean(fit_A, axis=0)
tevalues = np.mean(te_A, axis=0)

fit_ax = ax[0][0]
te_ax = ax[0][1]

im_fit = fit_ax.imshow(fitvalues, origin='lower', cmap='YlGn')
im_te = te_ax.imshow(tevalues, origin='lower', cmap='RdPu')

fit_ax.set_title('FIT', fontsize=22)
te_ax.set_title('TE', fontsize=22)

cbar_te = te_ax.figure.colorbar(im_te, ax = te_ax)
cbar_fit = te_ax.figure.colorbar(im_fit, ax = fit_ax)

cbar_fit.set_label(label='bits', fontsize=20)
cbar_fit.ax.tick_params(labelsize = 18)

cbar_te.set_label(label='bits', fontsize=20)
cbar_te.ax.tick_params(labelsize = 18)

fit_ax.set_xlabel("$w_{noise}$", fontsize=20)
fit_ax.set_ylabel("$w_{stim}$", fontsize=20)

te_ax.set_xlabel("$w_{noise}$", fontsize=20)
te_ax.set_ylabel("$w_{stim}$", fontsize=20)

fit_ax.tick_params(axis='both', which='major', labelsize=16)
te_ax.tick_params(axis='both', which='major', labelsize=16)

te_ax.set_xlabel("$w_{noise}$")
te_ax.set_ylabel("$w_{stim}$")

plt.suptitle("FIT and TE as a function of stimulus transmission strenght", fontsize = 24)
plt.show()

In [ ]:
# 2B plot

te_B_avg = np.mean(te_B[0:2], axis=(0,2))
fit_B_avg = np.mean(fit_B[0:2], axis=(0,2))

# error associated to the mean
te_B_std = np.std(te_B[0:2], axis=(0,2))
fit_B_std = np.std(fit_B[0:2], axis=(0,2))
frac = 1/np.sqrt(50 * 2000)
te_sem = te_B_std * frac
fit_sem = fit_B_std * frac

fig, ax = plt.subplots(figsize=(10,6))
ax.grid(alpha=0.4)

ax1 = ax.twinx()

ax.plot(fit_B_avg, label='FIT', markersize=3)
ax1.plot(te_B_avg, label='TE', markersize=3)

# ax.fill_between()

ax.legend()

# plt.axvspan(33, 40, color='red', alpha=0.3)

plt.show()
